In [65]:
# coding: utf-8
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import jieba.posseg
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_excel('货物分类训练数据.xlsx')
pre = pd.read_excel('需要分类货物数据 .xlsx')

train = train.drop_duplicates()
train=train.reset_index(drop=True)
# train = train[1:1000]
# print(train.shape)


def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False


def format_str(content):
    content_str = ''
    for i in content:
        if is_chinese(i):
            content_str = content_str + i
    return content_str

train['sort_name'] = train['货物全称'].apply(lambda x: format_str(x))
pre['sort_name'] = pre['货物名称'].apply(lambda x: format_str(x))

train = train[train['一级分类'] != '待定']
train=train.reset_index(drop=True)

def clean(x):
    x = x.replace("其他", "其它")
    return x


train['一级分类'] = train['一级分类'].apply(lambda x: clean(x))


def my_test(a,b,c):
    if b != b or b == '\\':
        return False
    if a != a or a == '\\':
        return False
    if c != c or c == '\\':
        return False
    return True

train['is_right'] = train.apply(lambda x: my_test(x['一级分类'],x['二级分类'], x['三级分类']), axis=1)
train=train[train['is_right']==True]
train=train.reset_index(drop=True)

# 思路：多标签、多种类
# 监督学习：分词+相似度匹配+K近邻？分词+文本特征+训练（这个因为标签太多很难多分类，有的只有1种）？

# 先直接分词，然后实验算法（随机划分，先打好基础，观察准确率） 一级二级与三级


import jieba
from sklearn import preprocessing

# le = preprocessing.LabelEncoder()
# le.fit(train['一级分类'])
# train['1_kind'] = le.transform(train['一级分类'])
# le.fit(train['二级分类'].astype(str))
# train['2_kind'] = le.transform(train['二级分类'].astype(str))
# le.fit(train['三级分类'].astype(str))
# train['3_kind'] = le.transform(train['三级分类'].astype(str))

# count1 = pd.DataFrame(train.groupby(['1_kind'])['一级分类'].count().reset_index())
# count1.columns = ['1_kind', 'count1']
# train = train.merge(count1, on='1_kind', how='left')
# count1 = pd.DataFrame(train.groupby(['2_kind'])['一级分类'].count().reset_index())
# count1.columns = ['2_kind', 'count2']
# train = train.merge(count1, on='2_kind', how='left')
# count1 = pd.DataFrame(train.groupby(['3_kind'])['一级分类'].count().reset_index())
# count1.columns = ['3_kind', 'count3']
# train = train.merge(count1, on='3_kind', how='left')

# train=train.reset_index(drop=True)

jieba.add_word('热镀')
jieba.add_word('方管')
jieba.add_word('盘螺') 
jieba.add_word('乌冬面')
jieba.add_word('栈板')
jieba.add_word('喷塑')
jieba.add_word('潘婷')
jieba.add_word('碧浪') 
jieba.add_word('汰渍')
jieba.add_word('麦格尼飞')
jieba.add_word('卷板')
jieba.add_word('郎牌')
jieba.add_word('原纸') 
jieba.add_word('伸性纸')
jieba.add_word('版纸')
jieba.add_word('薄本纸')
jieba.add_word('淮盐牌')
jieba.add_word('化学浆') 
jieba.add_word('宝光')
jieba.add_word('恒大')
jieba.add_word('分路器')
jieba.add_word('管坯钢')


def all(x):
    alist=list(jieba.cut(x,cut_all=True))
    blist=list(x)
    last=[]
#     a=list(jieba.posseg.cut(x))
# # #     print(a)
# #     for i in a:
# #         print(i.word,i.flag)
# # #     print(a)
# #     print(i,a)
#     try:
#         last=[[i.word for i in a if 'n' in i.flag][-1]]
#     except:
#         print(a)
#         last=[a[-1].word]

    return alist+blist+last

train=train.reset_index(drop=True)    
# train['word_cut']=train['sort_name'].apply(lambda x:list(jieba.cut(x,cut_all=True)))
# train['word_cut']=train['word_cut'].apply(lambda x:x+list(train['sort_name']))
train['word_cut'] = train['sort_name'].apply(lambda x: all(x))
pre['word_cut'] = pre['sort_name'].apply(lambda x: all(x))
# for i in train_many['sort_name']:
#     a=list(jieba.posseg.cut(i))
# #     print(a)
# #     print(i,list(jieba.posseg.cut(i)))
#     print([i.word for i in a if 'n' in i.flag][-1])
# vectorizer = CountVectorizer()
# #计算个词语出现的次数
# X = vectorizer.fit_transform(corpus)
allword=[]

for i in train.word_cut:
    for j in i:
        allword.append(j)
for i in pre.word_cut:
    for j in i:
        allword.append(j)
allword=list(set(allword))

def encode(x):
    encodelist=[True if i in x else False for i in allword]
#     lastlist=[True if i in x[-1] else False for i in allword]
    lastlist=[]
    return encodelist+lastlist
train['word_dict'] = train['word_cut'].apply(lambda x:encode(x))
train['word_dict']=train['word_dict'].apply(lambda x:list(x))
pre['word_dict'] = pre['word_cut'].apply(lambda x:encode(x))
pre['word_dict']=pre['word_dict'].apply(lambda x:list(x))
alist=[]
for i in train['word_dict']:
    alist.append(i)
# for i in pre['word_dict']:
#     alist.append(i)
wordarray=np.array(alist)
# import numpy as np
# a=np.array(train_many['word_cut'])
tree = neighbors.BallTree(wordarray,metric='sokalsneath')
al=[]
bl=[]
cl=[]
dl=[]
el=[]
fl=[]
count = 0
# train_many = pd.DataFrame(train[train['count3'] != 1])
# # train_many=train_many.reset_index(drop=True)
# # train_many.to_csv('aaaa.csv')
# for m in train_many['word_dict']:
#     dist, ind = tree.query(np.array(m).reshape(1, -1), k=4)
# #     print('dis:',dist)
#     print(ind)
#     a = train['三级分类'][ind[0][1]]
# #     b = train['三级分类'][ind[0][2]]
# #     e=train['三级分类'][ind[0][3]]
#     c = train['word_cut'][ind[0][1]]
# #     d = train['word_cut'][ind[0][2]]
# #     f=train['word_cut'][ind[0][3]]
#     al.append(a)
# #     bl.append(b)
# #     cl.append(c)
# #     el.append(e)
# #     fl.append(f)
#     cl.append(c)
# #     dl.append(d)

#     count=count+1
#     if count%100==0:
#         print(count)


In [66]:
# pre = pd.read_excel('需要分类货物数据.xlsx')
# train_many=train_many.reset_index(drop=True)
# train_many.to_csv('aaaa.csv')
for m in pre['word_dict']:
    dist, ind = tree.query(np.array(m).reshape(1, -1), k=1)
    print('dis:',dist)
#     print(ind)
    a3 = train['三级分类'][ind[0][0]]
    a2 = train['二级分类'][ind[0][0]]
    a1 = train['一级分类'][ind[0][0]]
#     b = train['三级分类'][ind[0][2]]
#     e=train['三级分类'][ind[0][3]]
#     c = train['word_cut'][ind[0][0]]
#     d = train['word_cut'][ind[0][2]]
#     f=train['word_cut'][ind[0][3]]
    al.append(a1)
    bl.append(a2)
    cl.append(a3)
#     bl.append(b)
#     cl.append(c)
#     el.append(e)
#     fl.append(f)
#     cl.append(c)
#     dl.append(d)

    count=count+1
    if count%100==0:
        print(count)

[[11489]]
[[10843]]
[[3915]]
[[1924]]
[[11489]]
[[10253]]
[[10256]]
[[11489]]
[[10478]]
[[11489]]
[[3771]]
[[11158]]
[[3893]]
[[11165]]
[[12011]]
[[4034]]
[[4023]]
[[12]]
[[11487]]
[[548]]
[[3915]]
[[10368]]
[[10445]]
[[1944]]
[[3918]]
[[12426]]
[[11900]]
[[10368]]
[[11914]]
[[10253]]
[[65]]
[[11167]]
[[11476]]
[[12449]]
[[11558]]
[[3449]]
[[10292]]
[[10478]]
[[11489]]
[[3879]]
[[155]]
[[11489]]
[[11332]]
[[1956]]
[[10774]]
[[12113]]
[[11723]]
[[3846]]
[[3522]]
[[11855]]
[[11489]]
[[3612]]
[[12164]]
[[3900]]
[[3832]]
[[0]]
[[11453]]
[[11898]]
[[11558]]
[[11031]]
[[3642]]
[[11273]]
[[12417]]
[[3898]]
[[11508]]
[[10718]]
[[10631]]
[[1927]]
[[11524]]
[[10975]]
[[10731]]
[[3848]]
[[11487]]
[[10485]]
[[10633]]
[[11489]]
[[3489]]
[[10494]]
[[12418]]
[[11489]]
[[1927]]
[[3489]]
[[10660]]
[[10459]]
[[11892]]
[[10715]]
[[3492]]
[[12116]]
[[11476]]
[[3830]]
[[1930]]
[[11892]]
[[3443]]
[[12417]]
[[11892]]
[[12417]]
[[1926]]
[[3879]]
[[10459]]
[[11273]]
100
[[10256]]
[[11476]]
[[3653]]
[[1960]]
[[

[[1429]]
[[10445]]
[[1977]]
[[690]]
[[803]]
[[395]]
[[10505]]
[[11502]]
[[690]]
[[395]]
[[10503]]
[[407]]
[[395]]
[[268]]
[[1926]]
[[593]]
[[1429]]
[[1429]]
[[690]]
[[11837]]
[[11489]]
[[11095]]
[[1429]]
[[1429]]
[[11489]]
[[11676]]
[[395]]
900
[[11489]]
[[1977]]
[[1429]]
[[1429]]
[[1429]]
[[3623]]
[[11489]]
[[3657]]
[[3449]]
[[1977]]
[[1977]]
[[1429]]
[[1956]]
[[1977]]
[[395]]
[[1977]]
[[12482]]
[[1994]]
[[1994]]
[[690]]
[[690]]
[[1994]]
[[10505]]
[[11875]]
[[268]]
[[1429]]
[[395]]
[[1429]]
[[395]]
[[0]]
[[11433]]
[[268]]
[[1977]]
[[395]]
[[1429]]
[[268]]
[[11489]]
[[11840]]
[[1977]]
[[1977]]
[[1429]]
[[3935]]
[[12449]]
[[3661]]
[[3661]]
[[3832]]
[[11840]]
[[11424]]
[[11718]]
[[1429]]
[[4017]]
[[11923]]
[[1994]]
[[1994]]
[[1977]]
[[1977]]
[[268]]
[[1977]]
[[1977]]
[[1994]]
[[11875]]
[[1429]]
[[1977]]
[[1977]]
[[1429]]
[[1977]]
[[1429]]
[[1977]]
[[1977]]
[[1994]]
[[1994]]
[[1977]]
[[1977]]
[[1977]]
[[1429]]
[[1429]]
[[1977]]
[[690]]
[[4131]]
[[268]]
[[268]]
[[690]]
[[10846]]
[[395]]
[[

[[220]]
[[268]]
[[11330]]
[[11330]]
1800
[[11782]]
[[11489]]
[[11527]]
[[11489]]
[[11405]]
[[11502]]
[[11523]]
[[268]]
[[11487]]
[[395]]
[[10484]]
[[268]]
[[3967]]
[[3967]]
[[1926]]
[[3661]]
[[11489]]
[[75]]
[[75]]
[[10481]]
[[268]]
[[268]]
[[3914]]
[[75]]
[[75]]
[[75]]
[[75]]
[[75]]
[[75]]
[[220]]
[[11495]]
[[690]]
[[690]]
[[395]]
[[690]]
[[690]]
[[395]]
[[11130]]
[[3967]]
[[10730]]
[[10486]]
[[10505]]
[[408]]
[[12136]]
[[1429]]
[[3637]]
[[59]]
[[4131]]
[[395]]
[[268]]
[[268]]
[[3832]]
[[395]]
[[11867]]
[[690]]
[[690]]
[[11523]]
[[268]]
[[268]]
[[268]]
[[268]]
[[268]]
[[268]]
[[10505]]
[[11867]]
[[10529]]
[[279]]
[[11489]]
[[3661]]
[[11433]]
[[3956]]
[[11495]]
[[395]]
[[395]]
[[3956]]
[[3661]]
[[395]]
[[395]]
[[395]]
[[11489]]
[[169]]
[[10736]]
[[690]]
[[10374]]
[[268]]
[[3908]]
[[11433]]
[[4131]]
[[268]]
[[268]]
[[268]]
[[268]]
[[268]]
[[3661]]
[[3661]]
[[10486]]
[[11521]]
[[11523]]
[[10736]]
[[268]]
1900
[[268]]
[[4131]]
[[268]]
[[11837]]
[[395]]
[[268]]
[[268]]
[[268]]
[[268]]
[[26

[[10445]]
[[10362]]
[[10771]]
[[10445]]
[[10445]]
[[11873]]
[[11489]]
[[11951]]
[[11198]]
[[155]]
[[11489]]
[[1940]]
[[11418]]
[[11132]]
[[11951]]
[[11586]]
[[268]]
[[604]]
[[3526]]
[[10445]]
[[3887]]
[[3887]]
[[3887]]
[[10766]]
[[1429]]
[[11718]]
[[11955]]
[[3887]]
[[3887]]
[[11049]]
[[10362]]
[[3867]]
[[157]]
[[1965]]
[[1940]]
[[4111]]
[[311]]
[[12566]]
[[10789]]
[[12523]]
[[11228]]
[[12523]]
[[12523]]
[[3887]]
[[10362]]
[[12080]]
[[10848]]
[[10848]]
[[10848]]
[[10848]]
[[11130]]
[[11130]]
[[3631]]
[[10362]]
[[11881]]
[[10445]]
[[10762]]
[[1005]]
[[10344]]
[[395]]
[[10848]]
[[395]]
[[10445]]
[[395]]
[[395]]
[[10848]]
[[11342]]
[[11424]]
[[10848]]
[[268]]
[[3887]]
[[395]]
[[311]]
[[11270]]
[[10737]]
[[10848]]
2800
[[10848]]
[[10848]]
[[10848]]
[[593]]
[[395]]
[[604]]
[[1956]]
[[11130]]
[[11130]]
[[10848]]
[[10848]]
[[11130]]
[[10848]]
[[10848]]
[[11130]]
[[10848]]
[[12142]]
[[1965]]
[[1312]]
[[1965]]
[[1965]]
[[358]]
[[395]]
[[1965]]
[[1312]]
[[1965]]
[[604]]
[[1965]]
[[3638]]
[[1965]

[[11602]]
[[11602]]
3600
[[11602]]
[[154]]
[[11602]]
[[11602]]
[[70]]
[[70]]
[[11602]]
[[11602]]
[[154]]
[[70]]
[[1826]]
[[11127]]
[[11127]]
[[11127]]
[[3748]]
[[3915]]
[[3915]]
[[11230]]
[[11315]]
[[1918]]
[[11230]]
[[11165]]
[[11784]]
[[3888]]
[[10977]]
[[3884]]
[[3902]]
[[10751]]
[[11142]]
[[3885]]
[[3748]]
[[11142]]
[[11124]]
[[3705]]
[[11353]]
[[3915]]
[[11149]]
[[3915]]
[[11145]]
[[3915]]
[[11159]]
[[3937]]
[[11159]]
[[3915]]
[[548]]
[[3748]]
[[11330]]
[[12492]]
[[3582]]
[[11923]]
[[12010]]
[[3225]]
[[10810]]
[[11718]]
[[11127]]
[[3748]]
[[11814]]
[[11462]]
[[12074]]
[[11140]]
[[11462]]
[[12152]]
[[11131]]
[[3915]]
[[11131]]
[[7]]
[[12104]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[7]]
[[3915]]
[[7]]
[[3748]]
[[7]]
[[7]]
[[3862]]
[[11315]]
[[3748]]
[[7]]
[[11928]]
[[3902]]
[[3902]]
[[3748]]
[[3884]]
[[3884]]
[[3884]]
[[3884]]
[[7]]
[[7]]
[[7]]
[[3884]]
[[803]]
3700
[[90]]
[[11353]]
[[3915]]
[[11230]]
[[3902]]
[[7]]
[[7]]
[[11127]]
[[7]]
[[3748]]
[[3748]]
[[374

[[11957]]
[[11109]]
[[11107]]
[[11110]]
[[12583]]
[[12484]]
[[11965]]
[[11446]]
[[11290]]
[[11700]]
[[11297]]
[[11276]]
[[3575]]
[[3729]]
[[12086]]
[[11763]]
[[12481]]
[[11069]]
[[3633]]
4500
[[11371]]
[[12097]]
[[11728]]
[[11727]]
[[12465]]
[[12464]]
[[10960]]
[[12094]]
[[11395]]
[[10636]]
[[12090]]
[[12089]]
[[11353]]
[[11353]]
[[74]]
[[11353]]
[[10907]]
[[11804]]
[[149]]
[[598]]
[[149]]
[[597]]
[[3536]]
[[3536]]
[[4104]]
[[4122]]
[[10906]]
[[4120]]
[[3897]]
[[11672]]
[[3536]]
[[11848]]
[[11350]]
[[3541]]
[[4119]]
[[12129]]
[[3700]]
[[10651]]
[[2]]
[[187]]
[[11279]]
[[65]]
[[10320]]
[[1931]]
[[1935]]
[[4117]]
[[3226]]
[[10466]]
[[3698]]
[[10474]]
[[6360]]
[[10476]]
[[12077]]
[[11837]]
[[3912]]
[[3923]]
[[3922]]
[[10504]]
[[4118]]
[[3916]]
[[10506]]
[[3561]]
[[3615]]
[[5]]
[[5]]
[[5]]
[[10451]]
[[11494]]
[[74]]
[[74]]
[[10733]]
[[5]]
[[5]]
[[11087]]
[[5]]
[[5]]
[[11330]]
[[10730]]
[[3513]]
[[5]]
[[5]]
[[11503]]
[[3650]]
[[10412]]
[[11501]]
[[5]]
[[5]]
[[5]]
[[10429]]
[[10529]]
[[12069

In [67]:
wordarray.shape

(12595, 4685)

In [68]:
train_many.shape

(12621, 15)

In [69]:
pre['一级分类']=al
pre['二级分类']=bl
pre['三级分类']=cl


# train_many['pre32']=bl
# train_many['w31']=cl
# train_many['w32']=dl
# train_many['pre33']=el
# train_many['w33']=fl
# train_many['pre33']=cl

In [70]:
# print((train_many[train_many['三级分类'] == train_many['pre31']].shape[0]) / train_many.shape[0]) 
# print((train_many[train_many['三级分类'] == train_many['pre32']].shape[0]) / train_many.shape[0]) 
# print((train_many[train_many['三级分类'] == train_many['pre33']].shape[0]) / train_many.shape[0]) 
# print((train_many[train_many['三级分类'] == train_many['pre33']].shape[0]) / train_many.shape[0]) # 正确率

# print(train_many[train_many['三级分类'] != train_many['pre3']])

In [71]:
# train_many['right1']=(train_many['三级分类'] == train_many['pre31'])
# train_many['right2']=(train_many['三级分类'] == train_many['pre32'])
# train_many['right3']=(train_many['三级分类'] == train_many['pre33'])
# train_many['rightany']=train_many['right1']|train_many['right2']|train_many['right3']

# def my_test(a, b,c):
#     if b==c:
#         return b
#     else:
#         return a


# train_many['pre123']=train_many.apply(lambda x: my_test(x['pre31'], x['pre32'], x['pre33']), axis=1)

# df['Value'] = df.apply(lambda row: my_test(row['a'], row['c']), axis=1)

In [72]:
# print((train_many[train_many['三级分类'] == train_many['pre123']].shape[0]) / train_many.shape[0]) #确实采用后两个的有提升

0.9256794231835829  S  4 投票
0.9286110450835908  S  1

0.7402022756005057 字  jaccard

0.7331700379266751  cut all=False jacc

0.747629582806574  cut all=True  jacc

0.7575853350189633  字+True  jacc

0.7492888748419722  字+False jacc

0.7540297092288243 字+True+后面两个字  matching 

0.7583754740834386  字+True+后面一个字  jacc

0.8424462705436156  字+True+后面一个字  jacc  knn=4 max?

0.9245701608430393
字+True+True+后面一个字  jacc  knn=2

0.9249663259646621
# sokalsneath  0.9307503367403533 字+True  knn=4 不融合 #
 0.9303541716187307 字+True  jacc knn=4 不融合 
目前改进方法：
利用词性+名词尝试
调参：距离 与 K tfidf?
集成
添加人工词典


想法:重点突出名词 实现方式：tf加其他相似度指标
或者先用其他监督学习训练大分类（比较小分类（一级的）准确度，最后KNN小分类）

In [87]:
del pre['word_dict']
del pre['sort_name']
del pre['word_cut']
pre.to_csv('pre_result.csv',index=False)
# pre.to_csv('pre_result.csv',encoding='gbk')

In [74]:
# diff=train_many[train_many['三级分类'] != train_many['pre31']]

In [75]:
# diff.shape

In [76]:
# diff.to_csv('aa.csv',encoding='gbk')

In [77]:
# diff[['right1','right2','right3','rightany']]#最后再尝试吧

In [78]:
# lena=[len(i) for i in al]

In [79]:
# max(lena)

In [80]:
# diff

In [81]:
# import jieba.posseg
# a=list(jieba.posseg.cut('漂亮的太阳牌的小玩具'));a

In [82]:
# for i in train_many['sort_name']:
#     a=list(jieba.posseg.cut(i))
# #     print(a)
# #     print(i,list(jieba.posseg.cut(i)))
#     print([i.word for i in a if 'n' in i.flag][-1])

In [83]:
# a[0].word

In [84]:
# a[0].flag